# MAS 4106 Final Project

**Authors**: Benton Stacy (bmstacy7127@eagle.fgcu.edu) and Katarya Johnson-Williams (kajohnsonwilliam3168@eagle.fgcu.edu)

**Abstract**:
The reason for writing this report is to explore the possible ways to predict years of education using census data. The processes and methods developed in this report could be applied to future census data to create an effective method of predicting significant census attributes. The problem this report seeks to solve is identifying patterns in the data provided by the census that can be used to predict years of education an individual has completed. A secondary problem the methodology in this report can be used to solve is to fill in missing data points in the census data. This report creates ten models that analyze the 15 different attributes included with the data in addition to adding two additional attributes (gross domestic product and Human Development Index for native countries). The models use a least squares solution which is validated by cross-validation. The results of this report did not find high variance in the error of each model but did make interesting discoveries regarding the relationship between the attributes indicating education and the native country of an individual. The overall conclusion made is that the United States population is significantly diverse and therefore it is difficult to accurately predict values such as years of education. The results of this report leave many avenues for future work including applying the models created to modern census data or adding additional census attributes (i.e. number of children or parental education levels) to see how these attributes affect the model. 

## Initialize the Dataset

In [1]:
# Load packages
using CSV, DataFrames, LinearAlgebra, Missings # For dataset loading and processing, running least-squares calculations.
using Plots, Measures, StatsPlots # For plotting results. Measures used for better plot margins. 
using Random # For generating random permutations.
using LaTeXStrings # For generating beautiful TeX-like equations in plots.

# Set up a few variables for the filepath of this file.
# NOTE: You may need to change this directory based on where your dataset is stored.
    # By default, it assumes the dataset is stored in (root)\Data Sets\Adult
root = dirname(@__FILE__) 
filename = "adult_full.csv"; 
filepath = joinpath(root, "Data Sets", "Adult", filename); 

# Read the dataset from the filepath.
adultDataSet = CSV.read(filepath,DataFrame;header=true, missingstring="?"); 
# There aren't too many missing values, so drop them.
adultDataSet = dropmissing(adultDataSet, disallowmissing=true) 

# Number of entries in the dataset
m = length(adultDataSet[:,1])

45222

## Construction of the Columns

The original dataset, after removing missing entries, has 45222 entries and 15 attributes, namely:

1. Age – age of the individual. This data is numerical, in whole numbers of years.
2. Work Class – sector the individual works in (government, private, self-employed, not working). The mode of this attribute was private, with nearly 75% of the entries.
3. Final Weight – a (continuous) number estimating the number of people in the United States population which matches the individual’s demographics. This value considers the potential for sampling bias in the census and assigns a "weight" to the person based on their demographics. Final weight may be calculated at the federal, state, or local level and thus values in this category may not be consistent.
4. Education – label describing the highest level of education completed. 
5. Education Number – arbitrary number assigned according to education label. 
6. Marital Status – current marital status of the individual (single, married, previously married).
7. Occupation – a variety of labels of the general job title of the individual. 
8. Relationship – a label of the relationship the individual has (wife, husband, unmarried, child, not in family).
9. Race – the race of the individual (white, Asian/Pacific Islander, Native American (“American Indian/Eskimo” in the data), black, other). 
10. Sex – the sex of the individual (male, female).
11. Capital Gain – a continuous category which measures an individual’s capital gain. Note that this is different from income and might include income earned via stocks and investments.
12. Capital Loss – a continuous category which measures an individual’s capital loss.
13. Hours Worked per Week – a numerical category which details the number of hours per week an individual works. 
14. Native Country – a label describing an individual’s native country. This attribute is highly skewed, with the mode of United States comprising of over 91% of the data.
15. Income Level – a Boolean (true/false) attribute which describes if the individual made a yearly income of greater than $50,000.


To process the data for purposes of creating and testing the models, the columns were modified as follows:
* Removed attributes:
    * Final weight – found to be inconsistently calculated across censuses, states, and local jurisdictions. In addition, information on final weight in the dataset documentation lacked sufficient detail.
    * Capital gain and capital loss – the original dataset documentation lacked sufficient detail on these columns. Also, theese attributes are *highly* skewed towards zero, thus providing little prediction power.
    * Education number – the original values seemed arbitrary. These values were replaced with a custom scale.
    * Relationship – little detail found in the original documentation. Similar in nature to the marital status attribute so that was used instead.
* Modified attributes:
    * Work class – separated into three columns based on the economic sector the individual is employed in: private sector, self-employed, or public (government). A fourth implicit column is used for those not working.
    * Education – the values were replaced with a scale based on the typical number of years an individual of that category would be expected to have completed. For example, a value of `10th` likely indicates around 11 years of schooling.
    * Marital status – labels were separated into two columns to represent single and married individuals. An implicit third column is used for those who have been previously married.
    * Occupation – labels were assigned one of five general occupational categories: engineering, business, technical, non-degree, and governmental jobs. The first four categories were assigned a column and the last one (government) was implicit.
    * Race – labels were assigned into four separate columns: white, Asian / Pacific-Islander, Native American (represented as `Amer-Indian-Eskimo`), and black. A fifth implicit column is designated fore those with the race label `other`.
    * Sex – transformed from a boolean variable into a numnerical one. Male is represented as "0" and female is represented as "1".
    * Native country – labels were assigned one of four categories based on the continent the country is located in: North America, South America, Asia, and Europe. The "Europe" column is implicit.
* Added attributes:
    * [Gross domestic product (GDP)](https://countryeconomy.com/gdp?year=1994) – a continuous value which represents the market value of the country's total goods and services produced over the year. The column consists of each applicable native country's 1994 GDP in $ millions USD, not accounting for inflation. GDP can be used as a secondary economic indicator for an individual by proxy of their native country.
    * [Human development index (HDI)](https://hdr.undp.org/data-center/human-development-index#/indicies/HDI) – an aggregate value on a continuous scale from 0 to 1, where higher values represent a higher level of human development. The formula for HDI calculations involve expected education levels, the country's GDP, and life expectancy. 1990 HDI data was extracted for the purposes of this program.
    
The revised dataset has 12 attributes. Attributes of more than one column have their columns listed. Implicit columns are *italicized*.

1. Age
2. Work Class
    - Private
    - Self-employed
    - Government
    - *Not working*
3. Education (years)
4. Marital Status
    - Single
    - Married
    - *Previously married*
5. Occupation
    - Engineering
    - Business
    - Technical
    - Non-degree
    - *Government*
6. Race
    - White
    - Asian / Pacific-Islander
    - Native American
    - Black
    - *Other*
7. Sex (Boolean)
8. Hours worked per week
9. Native Country
    - North America
    - South America
    - Asia
    - *Europe*
10. Income (Boolean)
11. GDP of native country
12. HDI of native country

In [2]:
# Age 
ageClass = adultDataSet[:,1];

# Work Class 
    # 100: Private 
    # 010: Self-employed 
    # 001: Government 
    # 000: Not working
workClass1 = adultDataSet[:,2].== " Private"

workClass21 = adultDataSet[:,2].== " Self-emp-not-inc"
workClass22 = adultDataSet[:,2].== " Self-emp-inc"

workClass31 = adultDataSet[:,2].== " Federal-gov"
workClass32 = adultDataSet[:,2].== " Local-gov"
workClass33 = adultDataSet[:,2].== " State-gov"

workClassMatrix = [workClass1 workClass21+workClass22 workClass31+workClass32+workClass33];

# Education
eduClass1 = (adultDataSet[:,4].== " Doctorate") .* 24
eduClass2 = (adultDataSet[:,4].== " Masters") .* 19
eduClass3 = (adultDataSet[:,4].== " Bachelors") .*17
eduClass4 = (adultDataSet[:,4].== " Some-college") .*14
eduClass5 = (adultDataSet[:,4].== " HS-grad") .*13
eduClass6 = (adultDataSet[:,4].== " 12th") .*13
eduClass7 = (adultDataSet[:,4].== " 11th") .* 12
eduClass8 = (adultDataSet[:,4].== " 10th") .* 11
eduClass9 = (adultDataSet[:,4].== " 9th") .* 10
eduClass10 = (adultDataSet[:,4].== " 7th-8th") .* 8
eduClass11 = (adultDataSet[:,4].== " 5th-6th") .* 6
eduClass12 = (adultDataSet[:,4].== " 1st-4th") .* 3
eduClass13 = (adultDataSet[:,4].== " Preschool") 
eduClass14 = (adultDataSet[:,4].== " Prof-school") .* 15
eduClass15 = (adultDataSet[:,4].== " Assoc-acdm") .* 15
eduClass16 = (adultDataSet[:,4].== " Assoc-voc") .* 15

eduClass = (eduClass1+eduClass2+eduClass3+eduClass4+eduClass5+eduClass6+eduClass7+eduClass8+eduClass9+
    eduClass10+ eduClass11+eduClass12+eduClass13+eduClass14+eduClass15+eduClass16);

# Marital Status 
    # 10: Single 
    # 01: Married 
    # 00: Previously married

marryClass1 = adultDataSet[:,6].== " Never-married"
marryClass21 = adultDataSet[:,6].== " Married-civ-spouse"
marryClass22 = adultDataSet[:,6].== " Married-spouse-absent"

marryClass23 = adultDataSet[:,6].== " Married-AF-spouse"

marryMatrix = [marryClass1 marryClass21+marryClass22+marryClass23];

# Occupation 
    # 1000: Engineering 
    # 0100: Business 
    # 0010: Technical 
    # 0001: Non-degree 
    # 0000: Government

occClass11 = adultDataSet[:,7].== " Tech-support"
occClass12 = adultDataSet[:,7].== " Machine-op-inspct"
occClass21 = adultDataSet[:,7].== " Sales"
occClass22 = adultDataSet[:,7].== " Exec-managerial"
occClass23 = adultDataSet[:,7].== " Adm-clerical"
occClass31 = adultDataSet[:,7].== " Craft-repair"
occClass32 = adultDataSet[:,7].== " Prof-specialty"
occClass41 = adultDataSet[:,7].== " Other-service"
occClass42 = adultDataSet[:,7].== " Handlers-cleaners"
occClass43 = adultDataSet[:,7].== " Farming-fishing"
occClass44 = adultDataSet[:,7].== " Transport-moving"

occMatrix = [occClass11+occClass12 occClass21+occClass22+occClass23 occClass31+occClass32 occClass41+occClass42+
    occClass43+occClass44];

# Race
raceClass1 = adultDataSet[:,9].== " White"
raceClass2 = adultDataSet[:,9].== " Asian-Pac-Islander"
raceClass3 = adultDataSet[:,9].== " Amer-Indian-Eskimo"
raceClass4 = adultDataSet[:,9].== " Black"

raceMatrix = [raceClass1 raceClass2 raceClass3 raceClass4];

# Sex 
sexClass = adultDataSet[:,10].== " Female";

# Hours Per Week 
hrsPerWeekClass = adultDataSet[:,13];

# Native Country 
    # 100: North America 
    # 010: South America 
    # 001: Asia 
    # 000: Europe
natClass11 = adultDataSet[:,14].== " United-States"
natClass12 = adultDataSet[:,14].== " Outlying-US(Guam-USVI-etc)"
natClass13 = adultDataSet[:,14].== " Puerto-Rico"
natClass14 = adultDataSet[:,14].== " Canada"
natClass15 = adultDataSet[:,14].== " Cuba"
natClass16 = adultDataSet[:,14].== " Honduras"
natClass17 = adultDataSet[:,14].== " Jamaica"
natClass18 = adultDataSet[:,14].== " Mexico"
natClass19 = adultDataSet[:,14].== " Dominican-Republic"
natClass110 = adultDataSet[:,14].== " Haiti"
natClass111 = adultDataSet[:,14].== " Nicaragua"
natClass112 = adultDataSet[:,14].== " El-Salvador"
natClass113 = adultDataSet[:,14].== " Trinidad&Tobago"

natClass21 = adultDataSet[:,14].== " Ecuador"
natClass22 = adultDataSet[:,14].== " Columbia"
natClass23 = adultDataSet[:,14].== " Guatemala"
natClass24 = adultDataSet[:,14].== " Peru"

natClass31 = adultDataSet[:,14].== " Cambodia"
natClass32 = adultDataSet[:,14].== " India"
natClass33 = adultDataSet[:,14].== " Japan"
natClass34 = adultDataSet[:,14].== " South"
natClass35 = adultDataSet[:,14].== " China"
natClass36 = adultDataSet[:,14].== " Iran"
natClass37 = adultDataSet[:,14].== " Philippines"
natClass38 = adultDataSet[:,14].== " Vietnam"
natClass39 = adultDataSet[:,14].== " Laos"
natClass310 = adultDataSet[:,14].== " Taiwan"
natClass311 = adultDataSet[:,14].== " Thailand"
natClass312 = adultDataSet[:,14].== " Hong"

natMatrix = [(natClass11+natClass12+natClass13+natClass14+natClass15+natClass16+natClass17+natClass18+natClass19+
        natClass110+natClass111+natClass112+natClass113) (natClass21+natClass22+natClass23+natClass24) (natClass31+
        natClass32+natClass33+natClass34+natClass35+natClass36+natClass37+natClass38+natClass39+natClass310+
        natClass311+natClass312)];

# Native Country GDP 
# Numbers are in Millions of USD
# GDP data from 1994
# Source: https://countryeconomy.com/gdp?year=1994
gdpClass1 = (adultDataSet[:,14].== " United-States") .* 7287200
gdpClass2 = (adultDataSet[:,14].== " Outlying-US(Guam-USVI-etc)") .* 7287200 # Uses US GDP
gdpClass3 = (adultDataSet[:,14].== " Puerto-Rico") .* 7287200 # Uses US GDP
gdpClass4 = (adultDataSet[:,14].== " Canada") .* 579913
gdpClass5 = (adultDataSet[:,14].== " Cuba") .* 28448
gdpClass6 = (adultDataSet[:,14].== " Honduras") .* 4642
gdpClass7 = (adultDataSet[:,14].== " Jamaica") .* 5453
gdpClass8 = (adultDataSet[:,14].== " Mexico") .* 527811
gdpClass9 = (adultDataSet[:,14].== " Dominican-Republic") .* 14645
gdpClass10 = (adultDataSet[:,14].== " Haiti") .* 3054
gdpClass11 = (adultDataSet[:,14].== " Nicaragua") .* 3861
gdpClass12 = (adultDataSet[:,14].== " El-Salvador") .* 7679
gdpClass13 = (adultDataSet[:,14].== " Trinadad&Tobago") .* 5032
gdpClass14 = (adultDataSet[:,14].== " Ecuador") .* 21147
gdpClass15 = (adultDataSet[:,14].== " Columbia") .* 97625
gdpClass16 = (adultDataSet[:,14].== " Guatemala") .* 12501
gdpClass17 = (adultDataSet[:,14].== " Peru") .* 43225
gdpClass18 = (adultDataSet[:,14].== " Cambodia") .* 2765
gdpClass19 = (adultDataSet[:,14].== " India") .* 333014
gdpClass20 = (adultDataSet[:,14].== " Japan") .* 4998797
gdpClass21 = (adultDataSet[:,14].== " South") .* 463520
gdpClass22 = (adultDataSet[:,14].== " China") .* 561686
gdpClass23 = (adultDataSet[:,14].== " Iran") .* 79818
gdpClass24 = (adultDataSet[:,14].== " Philippines") .* 73159
gdpClass25 = (adultDataSet[:,14].== " Vietnam") .* 20712
gdpClass26 = (adultDataSet[:,14].== " Laos") .* 3081
gdpClass27 = (adultDataSet[:,14].== " Taiwan") .* 256247
gdpClass28 = (adultDataSet[:,14].== " Thailand") .* 146684
gdpClass29 = (adultDataSet[:,14].== " Hong") .* 135812
gdpClass30 = (adultDataSet[:,14].== " England") .* 1244009
gdpClass31 = (adultDataSet[:,14].== " Germany") .* 2209934
gdpClass32 = (adultDataSet[:,14].== " Greece") .* 115694
gdpClass33 = (adultDataSet[:,14].== " Italy") .* 1088506
gdpClass34 = (adultDataSet[:,14].== " Poland") .* 103887
gdpClass35 = (adultDataSet[:,14].== " Portugal") .* 99692
gdpClass36 = (adultDataSet[:,14].== " Ireland") .* 55843
gdpClass37 = (adultDataSet[:,14].== " France") .* 1396653
gdpClass38 = (adultDataSet[:,14].== " Hungary") .* 43167
gdpClass39 = (adultDataSet[:,14].== " Scotland") .* 1244009 # Uses England's GDP
# Yugoslavia was split into Bosnia and Herzegovina, Croatia, Macedonia, Montenegro, 
# Serbia, Slovenia, and Kosovo (not in data) in 1991 so these GDPs were all added together
gdpClass40 = (adultDataSet[:,14].== " Yugoslavia") .* 60158
gdpClass41 = (adultDataSet[:,14].== " Holand-Netherlands") .* 382550

gdpClass = (gdpClass1+gdpClass2+gdpClass3+gdpClass4+gdpClass5+gdpClass6+gdpClass7+gdpClass8+gdpClass9+gdpClass10+
    gdpClass11+gdpClass12+gdpClass13+gdpClass14+gdpClass15+gdpClass16+gdpClass17+gdpClass18+gdpClass19+gdpClass20+
    gdpClass21+gdpClass22+gdpClass23+gdpClass24+gdpClass25+gdpClass26+gdpClass27+gdpClass28+gdpClass29+gdpClass30+
    gdpClass31+gdpClass32+gdpClass33+gdpClass34+gdpClass35+gdpClass36+gdpClass37+gdpClass38+gdpClass39+gdpClass40+
    gdpClass41);


# Native Country HDI
# HDI, or Human Development Index, is a statistic of human development, broken down into 
    # life expectancy;
    # education;
    # income.
# HDI data from 1990.
# Source: https://hdr.undp.org/data-center/documentation-and-downloads
hdiClass1 = (adultDataSet[:,14].== " United-States") .* 0.872
hdiClass2 = (adultDataSet[:,14].== " Outlying-US(Guam-USVI-etc)") .* 0.872 # Uses US HDI
hdiClass3 = (adultDataSet[:,14].== " Puerto-Rico") .* 0.872 # Uses US HDI
hdiClass4 = (adultDataSet[:,14].== " Canada") .* 0.860
hdiClass5 = (adultDataSet[:,14].== " Cuba") .* 0.680
hdiClass6 = (adultDataSet[:,14].== " Honduras") .* 0.516
hdiClass7 = (adultDataSet[:,14].== " Jamaica") .* 0.659
hdiClass8 = (adultDataSet[:,14].== " Mexico") .* 0.662
hdiClass9 = (adultDataSet[:,14].== " Dominican-Republic") .* 0.577
hdiClass10 = (adultDataSet[:,14].== " Haiti") .* 0.429
hdiClass11 = (adultDataSet[:,14].== " Nicaragua") .* 0.490
hdiClass12 = (adultDataSet[:,14].== " El-Salvador") .* 0.525
hdiClass13 = (adultDataSet[:,14].== " Trinadad&Tobago") .* 0.660
hdiClass14 = (adultDataSet[:,14].== " Ecuador") .* 0.651
hdiClass15 = (adultDataSet[:,14].== " Columbia") .* 0.610
hdiClass16 = (adultDataSet[:,14].== " Guatemala") .* 0.484
hdiClass17 = (adultDataSet[:,14].== " Peru") .* 0.621
hdiClass18 = (adultDataSet[:,14].== " Cambodia") .* 0.378
hdiClass19 = (adultDataSet[:,14].== " India") .* 0.434
hdiClass20 = (adultDataSet[:,14].== " Japan") .* 0.845
hdiClass21 = (adultDataSet[:,14].== " South") .* 0.737
hdiClass22 = (adultDataSet[:,14].== " China") .* 0.484
hdiClass23 = (adultDataSet[:,14].== " Iran") .* 0.601
hdiClass24 = (adultDataSet[:,14].== " Philippines") .* 0.598
hdiClass25 = (adultDataSet[:,14].== " Vietnam") .* 0.482
hdiClass26 = (adultDataSet[:,14].== " Laos") .* 0.405
hdiClass27 = (adultDataSet[:,14].== " Taiwan") .* 0.484 # Uses China's HDI
hdiClass28 = (adultDataSet[:,14].== " Thailand") .* 0.576
hdiClass29 = (adultDataSet[:,14].== " Hong") .* 0.788
hdiClass30 = (adultDataSet[:,14].== " England") .* 0.804
hdiClass31 = (adultDataSet[:,14].== " Germany") .* 0.829
hdiClass32 = (adultDataSet[:,14].== " Greece") .* 0.759
hdiClass33 = (adultDataSet[:,14].== " Italy") .* 0.778
hdiClass34 = (adultDataSet[:,14].== " Poland") .* 0.716
hdiClass35 = (adultDataSet[:,14].== " Portugal") .* 0.701
hdiClass36 = (adultDataSet[:,14].== " Ireland") .* 0.737
hdiClass37 = (adultDataSet[:,14].== " France") .* 0.791
hdiClass38 = (adultDataSet[:,14].== " Hungary") .* 0.720
hdiClass39 = (adultDataSet[:,14].== " Scotland") .* 0.804 # Uses England's HDI
# Yugoslavia was split into Bosnia and Herzegovina, Croatia, Macedonia, Montenegro, 
# Serbia, Slovenia, and Kosovo (not in data) in 1991 so these HDIs were (weighted per capita) averaged
hdiClass40 = (adultDataSet[:,14].== " Yugoslavia") .* 0.714
hdiClass41 = (adultDataSet[:,14].== " Holand-Netherlands") .* 0.847

hdiClass = (hdiClass1+hdiClass2+hdiClass3+hdiClass4+hdiClass5+hdiClass6+hdiClass7+hdiClass8+hdiClass9+hdiClass10+
    hdiClass11+hdiClass12+hdiClass13+hdiClass14+hdiClass15+hdiClass16+hdiClass17+hdiClass18+hdiClass19+hdiClass20+
    hdiClass21+hdiClass22+hdiClass23+hdiClass24+hdiClass25+hdiClass26+hdiClass27+hdiClass28+hdiClass29+hdiClass30+
    hdiClass31+hdiClass32+hdiClass33+hdiClass34+hdiClass35+hdiClass36+hdiClass37+hdiClass38+hdiClass39+hdiClass40+
    hdiClass41);


# Income 
incomeClass = adultDataSet[:,15].== " <=50K";

# Create Matrix 
adultMatrix = [ones(m) ageClass workClassMatrix marryMatrix occMatrix raceMatrix sexClass hrsPerWeekClass natMatrix gdpClass hdiClass incomeClass]
# display(adultMatrix)

45222×23 Matrix{Float64}:
 1.0  39.0  0.0  0.0  1.0  1.0  0.0  …  0.0  0.0       7.2872e6  0.872  1.0
 1.0  50.0  0.0  1.0  0.0  0.0  1.0     0.0  0.0       7.2872e6  0.872  1.0
 1.0  38.0  1.0  0.0  0.0  0.0  0.0     0.0  0.0       7.2872e6  0.872  1.0
 1.0  53.0  1.0  0.0  0.0  0.0  1.0     0.0  0.0       7.2872e6  0.872  1.0
 1.0  28.0  1.0  0.0  0.0  0.0  1.0     0.0  0.0   28448.0       0.68   1.0
 1.0  37.0  1.0  0.0  0.0  0.0  1.0  …  0.0  0.0       7.2872e6  0.872  1.0
 1.0  49.0  1.0  0.0  0.0  0.0  1.0     0.0  0.0    5453.0       0.659  1.0
 1.0  52.0  0.0  1.0  0.0  0.0  1.0     0.0  0.0       7.2872e6  0.872  0.0
 1.0  31.0  1.0  0.0  0.0  1.0  0.0     0.0  0.0       7.2872e6  0.872  0.0
 1.0  42.0  1.0  0.0  0.0  0.0  1.0     0.0  0.0       7.2872e6  0.872  0.0
 1.0  37.0  1.0  0.0  0.0  0.0  1.0  …  0.0  0.0       7.2872e6  0.872  0.0
 1.0  30.0  0.0  0.0  1.0  0.0  1.0     0.0  1.0  333014.0       0.434  0.0
 1.0  23.0  1.0  0.0  0.0  1.0  0.0     0.0  0.0       7.2872e

## Summary Visualizations

### Age

In [3]:
# Create the histogram for age data and save it
ageHist = histogram(adultDataSet[:, 1], bins=15, lab="Age", xlab="Age (years)", ylab="Frequency", 
    title="Histogram of Age Data  " * L"(n=%$m)", color=:blues)
savefig(ageHist, "Age_statistics.png")

"C:\\Users\\bento\\OneDrive\\School Documents\\Spring 2023\\MAS 4106\\Final Project\\Age_statistics.png"

### Work Class

In [4]:
# Create proprtions for the work class categories:
    # y[1]: Private 
    # y[2]: Self-employed 
    # y[3]: Government 
    # y[4]: Not working
y = zeros(4);
y[1] = count(workClassMatrix[:, 1].==1)/m;
y[2] = count(workClassMatrix[:, 2].==1)/m;
y[3] = count(workClassMatrix[:, 3].==1)/m;
y[4] = 1 - y[1] - y[2] - y[3];

# Set up data labels for the pie chart
x = ["Private", "Self-employed", "Government", "Not working"]

# Create the pie chart for work class data and save it
workClassPie = pie(x, y, title = "Work Class  " * L"(n=%$m)")
savefig(workClassPie, "Work_class_statistics.png")

"C:\\Users\\bento\\OneDrive\\School Documents\\Spring 2023\\MAS 4106\\Final Project\\Work_class_statistics.png"

### Native Country

In [5]:
# Create proportions for the native country categories:
    # y[1]: United States
    # y[2]: The rest of North America 
    # y[3]: South America 
    # y[4]: Asia 
    # y[5]: Europe
y = zeros(5);
y[1] = count(adultDataSet[:,14].== " United-States") / m
y[2] = count(natClass12+natClass13+natClass14+natClass15+natClass16+natClass17+natClass18+natClass19+
    natClass110+natClass111+natClass112+natClass113 .== 1) / m
y[3] = count(natClass21+natClass22+natClass23+natClass24 .== 1) / m
y[4] = count(natClass31+natClass32+natClass33+natClass34+natClass35+natClass36+natClass37+natClass38+
    natClass39+natClass310+natClass311+natClass312 .== 1) / m
y[5] = 1 - y[1] - y[2] - y[3] - y[4];

# Set up data labels for the pie chart
x = ["United States", "North America*", "South America", "Asia", "Europe"]
    
# Create the pie chart for native country data and save it
workClassPie = pie(x, y, title = "Native Country  " * L"(n=%$m)", color=["brown", "brown1", "chartreuse2", 
        "gold", "deepskyblue"])
annotate!(0.65, -1.1, [text("* The rest of North America, excluding the United States", 8)])
savefig(workClassPie, "Native_country_statistics.png")

"C:\\Users\\bento\\OneDrive\\School Documents\\Spring 2023\\MAS 4106\\Final Project\\Native_country_statistics.png"

## Exploratory Models

### Model 1: Age & Hours Worked Per Week

In [6]:
# Create the matrix for model 1
Amod1 = [ones(m) ageClass hrsPerWeekClass]

# Create a number storing the number of elements in each fold
fold = div(m,5); 

# Create a random permutation of m-values
I = Random.randperm(m);

# Storage for cross-validation results
xhats1 = zeros(3,5); 
rmsErrors1 = zeros(2,5);

# For each fold, compute the appropriate model and store its error.
anim = @animate for k = 1:5
    
    # Assign a random permutation into training and test data
    if (k == 1) 
        Itest = I[1 : fold]; 
        Itrain = I[fold+1 : end];
    elseif (k == 5)
        Itest = I[4 * fold+1 : end]; 
        Itrain = I[1:4 * fold];
    else 
        Itest = I[(k-1) * fold+1: k*fold];
        Itrain = I[[1 : (k-1) * fold ; k * fold + 1 : m]];
    end

    #display(Itest);
    #display(Itrain);
    
    # Compute sample sizes for training and test data
    mTest = length(Itest);
    mTrain = length(Itrain);

    # Compute the model based on training data and store it
    xhats1[:, k] = Amod1[Itrain, :] \ eduClass[Itrain];

    # Compute RMS errors for both training and test data
    rmsErrors1[1,k] = norm(Amod1[Itrain,:] * xhats1[:, k] - eduClass[Itrain]) / sqrt(mTrain);
    rmsErrors1[2,k] = norm(Amod1[Itest,:] * xhats1[:, k] - eduClass[Itest]) / sqrt(mTest);
    
    ## Plot actual (x) versus predicted (y) education
    # Plot test data
    tp1 = scatter(eduClass[Itest], Amod1[Itest, :] * xhats1[:, k], color="seagreen3", xlims=(0,25), ylims=(0,25), 
        title="Test data  " * L"(n=%$mTest)", xlabel="Actual education (years)", 
        ylabel="Predicted education (years)", minorgrid=true)
    plot!([0,25], [0,25], linestyle = :dash, linewidth=2, color="magenta") # y=x line
    
    # Plot training data
    p1 = scatter(eduClass[Itrain], Amod1[Itrain, :] * xhats1[:, k], color="seagreen3", xlims=(0,25), ylims=(0,25), 
        title="Training data  " * L"(n=%$mTrain)", xlabel="Actual education (years)", 
        ylabel="Predicted education (years)", minorgrid=true)
    plot!([0,25], [0,25], linestyle = :dash, linewidth=2, color="magenta") # y=x line

    plotModel1 = plot(tp1, p1, layout = (1, 2), legend=false, size=(1200, 620), margin=8mm,
        plot_title="Education Prediction using Age and Hours Worked per Week")
    annotate!(24, 1, [text("Fold " * string(k), 9)])
    savefig(plotModel1, "PlotModel_1f" * string(k) * ".png")
end

gif(anim, "PlotModel1.gif", fps=1)

println("x-hats: "); display(xhats1);
println("RMS errors: "); display(rmsErrors1);

x-hats: 


[ Info: Saved animation to C:\Users\bento\OneDrive\School Documents\Spring 2023\MAS 4106\Final Project\PlotModel1.gif


3×5 Matrix{Float64}:
 12.8221      12.8993      12.8792      12.888       12.9344
  0.00387641   0.00305791   0.00449042   0.00433938   0.00347041
  0.0294873    0.0280251    0.0276828    0.0275504    0.0272265

RMS errors: 


2×5 Matrix{Float64}:
 2.6819   2.67759  2.67193  2.67043  2.67833
 2.65295  2.67032  2.6927   2.69863  2.66716

### Model 2: Work Class & Occupation

In [7]:
# Create the matrix for model 2
Amod2 = [ones(m) workClassMatrix occMatrix]

# Create a number storing the number of elements in each fold
fold = div(m,5); 

# Create a random permutation of m-values
I = Random.randperm(m);

# Storage for cross-validation results
xhats2 = zeros(8,5); 
rmsErrors2 = zeros(2,5);

# For each fold, compute the appropriate model and store its error.
anim = @animate for k = 1:5
    
    # Assign a random permutation into training and test data
    if (k == 1) 
        Itest = I[1 : fold]; 
        Itrain = I[fold+1 : end];
    elseif (k == 5)
        Itest = I[4 * fold+1 : end]; 
        Itrain = I[1:4 * fold];
    else 
        Itest = I[(k-1) * fold+1: k*fold];
        Itrain = I[[1 : (k-1) * fold ; k * fold + 1 : m]];
    end

    #display(Itest);
    #display(Itrain);
    
    # Compute sample sizes for training and test data
    mTest = length(Itest);
    mTrain = length(Itrain);

    # Compute the model based on training data and store it
    xhats2[:, k] = Amod2[Itrain, :] \ eduClass[Itrain];

    # Compute RMS errors for both training and test data
    rmsErrors2[1,k] = norm(Amod2[Itrain,:] * xhats2[:, k] - eduClass[Itrain]) / sqrt(mTrain);
    rmsErrors2[2,k] = norm(Amod2[Itest,:] * xhats2[:, k] - eduClass[Itest]) / sqrt(mTest);
    
    ## Plot actual (x) versus predicted (y) education
    # Plot test data
    tp2 = scatter(eduClass[Itest], Amod2[Itest, :] * xhats2[:, k], color="firebrick1", xlims=(0,25), ylims=(0,25), 
        title="Test data  " * L"(n=%$mTest)", xlabel="Actual education (years)", 
        ylabel="Predicted education (years)", minorgrid=true)
    plot!([0,25], [0,25], linestyle = :dash, linewidth=2, color="magenta") # y=x line
    
    # Plot training data
    p2 = scatter(eduClass[Itrain], Amod2[Itrain, :] * xhats2[:, k], color="firebrick1", xlims=(0,25), ylims=(0,25), 
        title="Training data  " * L"(n=%$mTrain)", xlabel="Actual education (years)", 
        ylabel="Predicted education (years)", minorgrid=true)
    plot!([0,25], [0,25], linestyle = :dash, linewidth=2, color="magenta") # y=x line

    plotModel2 = plot(tp2, p2, layout = (1, 2), legend=false, size=(1200, 620), margin=8mm,
        plot_title="Education Prediction using Work Class and Occupation")
    annotate!(24, 1, [text("Fold " * string(k), 9)])
    savefig(plotModel2, "PlotModel_2f" * string(k) * ".png")
end

gif(anim, "PlotModel2.gif", fps=1)

println("x-hats: "); display(xhats2);
println("RMS errors: "); display(rmsErrors2);

x-hats: 


[ Info: Saved animation to C:\Users\bento\OneDrive\School Documents\Spring 2023\MAS 4106\Final Project\PlotModel2.gif


8×5 Matrix{Float64}:
 12.2886    12.5095    12.8449    12.7553    12.5436
  0.642657   0.442268   0.165306   0.281487   0.434281
  0.937226   0.774063   0.466275   0.592344   0.735789
  1.82084    1.59737    1.29116    1.42872    1.58777
  0.297428   0.289188   0.213166   0.207027   0.262904
  1.59621    1.55498    1.52847    1.47667    1.54411
  1.81704    1.86363    1.79118    1.74162    1.79647
 -0.293376  -0.327893  -0.391596  -0.413411  -0.366527

RMS errors: 


2×5 Matrix{Float64}:
 2.51346  2.49752  2.51356  2.49871  2.49845
 2.4684   2.53226  2.46769  2.5272   2.52817

### Model 3: Marital Status & Native Country

In [8]:
# Create the matrix for model 3
Amod3 = [ones(m) marryMatrix natMatrix]

# Create a number storing the number of elements in each fold
fold = div(m,5); 

# Create a random permutation of m-values
I = Random.randperm(m);

# Storage for cross-validation results
xhats3 = zeros(6,5); 
rmsErrors3 = zeros(2,5);

# For each fold, compute the appropriate model and store its error.
anim = @animate for k = 1:5
    
    # Assign a random permutation into training and test data
    if (k == 1) 
        Itest = I[1 : fold]; 
        Itrain = I[fold+1 : end];
    elseif (k == 5)
        Itest = I[4 * fold+1 : end]; 
        Itrain = I[1:4 * fold];
    else 
        Itest = I[(k-1) * fold+1: k*fold];
        Itrain = I[[1 : (k-1) * fold ; k * fold + 1 : m]];
    end

    #display(Itest);
    #display(Itrain);
    
    # Compute sample sizes for training and test data
    mTest = length(Itest);
    mTrain = length(Itrain);

    # Compute the model based on training data and store it
    xhats3[:, k] = Amod3[Itrain, :] \ eduClass[Itrain];

    # Compute RMS errors for both training and test data
    rmsErrors3[1,k] = norm(Amod3[Itrain,:] * xhats3[:, k] - eduClass[Itrain]) / sqrt(mTrain);
    rmsErrors3[2,k] = norm(Amod3[Itest,:] * xhats3[:, k] - eduClass[Itest]) / sqrt(mTest);
    
    ## Plot actual (x) versus predicted (y) education
    # Plot test data
    tp3 = scatter(eduClass[Itest], Amod3[Itest, :] * xhats3[:, k], color="olive", xlims=(0,25), ylims=(0,25), 
        title="Test data  " * L"(n=%$mTest)", xlabel="Actual education (years)", 
        ylabel="Predicted education (years)", minorgrid=true)
    plot!([0,25], [0,25], linestyle = :dash, linewidth=2, color="magenta") # y=x line
    
    # Plot training data
    p3 = scatter(eduClass[Itrain], Amod3[Itrain, :] * xhats3[:, k], color="olive", xlims=(0,25), ylims=(0,25), 
        title="Training data  " * L"(n=%$mTrain)", xlabel="Actual education (years)", 
        ylabel="Predicted education (years)", minorgrid=true)
    plot!([0,25], [0,25], linestyle = :dash, linewidth=2, color="magenta") # y=x line

    plotModel3 = plot(tp3, p3, layout = (1, 2), legend=false, size=(1200, 620), margin=8mm,
        plot_title="Education Prediction using Marital Status and Native Country")
    annotate!(24, 1, [text("Fold " * string(k), 9)])
    savefig(plotModel3, "PlotModel_3f" * string(k) * ".png")
end

gif(anim, "PlotModel3.gif", fps=1)

println("x-hats: "); display(xhats3);
println("RMS errors: "); display(rmsErrors3);

x-hats: 


[ Info: Saved animation to C:\Users\bento\OneDrive\School Documents\Spring 2023\MAS 4106\Final Project\PlotModel3.gif


6×5 Matrix{Float64}:
 13.899       13.9486       13.913      13.9106     13.842
  0.212644     0.151453      0.185883    0.155794    0.20564
  0.411226     0.364906      0.417396    0.393582    0.430863
  0.00042941   0.000927901  -0.0101563   0.0117566   0.0538128
 -1.96753     -1.74573      -1.97428    -1.69183    -1.82855
  1.0578       0.917736      1.06795     1.11464     1.08126

RMS errors: 


2×5 Matrix{Float64}:
 2.68102  2.6916   2.68673  2.67265  2.69284
 2.70123  2.65921  2.67841  2.73432  2.65371

### Model 4: Race & Sex

In [9]:
# Create the matrix for model 4
Amod4 = [ones(m) raceMatrix sexClass]

# Create a number storing the number of elements in each fold
fold = div(m,5); 

# Create a random permutation of m-values
I = Random.randperm(m);

# Storage for cross-validation results
xhats4 = zeros(6,5); 
rmsErrors4 = zeros(2,5);

# For each fold, compute the appropriate model and store its error.
anim = @animate for k = 1:5
    
    # Assign a random permutation into training and test data
    if (k == 1) 
        Itest = I[1 : fold]; 
        Itrain = I[fold+1 : end];
    elseif (k == 5)
        Itest = I[4 * fold+1 : end]; 
        Itrain = I[1:4 * fold];
    else 
        Itest = I[(k-1) * fold+1: k*fold];
        Itrain = I[[1 : (k-1) * fold ; k * fold + 1 : m]];
    end

    #display(Itest);
    #display(Itrain);
    
    # Compute sample sizes for training and test data
    mTest = length(Itest);
    mTrain = length(Itrain);

    # Compute the model based on training data and store it
    xhats4[:, k] = Amod4[Itrain, :] \ eduClass[Itrain];

    # Compute RMS errors for both training and test data
    rmsErrors4[1,k] = norm(Amod4[Itrain,:] * xhats4[:, k] - eduClass[Itrain]) / sqrt(mTrain);
    rmsErrors4[2,k] = norm(Amod4[Itest,:] * xhats4[:, k] - eduClass[Itest]) / sqrt(mTest);
    
    ## Plot actual (x) versus predicted (y) education
    # Plot test data
    tp4 = scatter(eduClass[Itest], Amod4[Itest, :] * xhats4[:, k], color="peru", xlims=(0,25), ylims=(0,25), 
        title="Test data  " * L"(n=%$mTest)", xlabel="Actual education (years)", 
        ylabel="Predicted education (years)", minorgrid=true)
    plot!([0,25], [0,25], linestyle = :dash, linewidth=2, color="magenta") # y=x line
    
    # Plot training data
    p4 = scatter(eduClass[Itrain], Amod4[Itrain, :] * xhats4[:, k], color="peru", xlims=(0,25), ylims=(0,25), 
        title="Training data  " * L"(n=%$mTrain)", xlabel="Actual education (years)", 
        ylabel="Predicted education (years)", minorgrid=true)
    plot!([0,25], [0,25], linestyle = :dash, linewidth=2, color="magenta") # y=x line

    plotModel4 = plot(tp4, p4, layout = (1, 2), legend=false, size=(1200, 620), margin=8mm,
        plot_title="Education Prediction using Race and Sex")
    annotate!(24, 1, [text("Fold " * string(k), 9)])
    savefig(plotModel4, "PlotModel_4f" * string(k) * ".png")
end

gif(anim, "PlotModel4.gif", fps=1)

println("x-hats: "); display(xhats4);
println("RMS errors: "); display(rmsErrors4);

x-hats: 


[ Info: Saved animation to C:\Users\bento\OneDrive\School Documents\Spring 2023\MAS 4106\Final Project\PlotModel4.gif


6×5 Matrix{Float64}:
 12.6183     12.7192     12.6062     12.6782     12.7709
  1.58615     1.48238     1.62069     1.55152     1.44552
  2.37436     2.36506     2.46029     2.38234     2.21216
  0.813531    0.774745    0.894445    0.865823    0.723098
  0.945584    0.887308    0.996028    0.93588     0.814597
  0.0841647   0.0409526   0.0449528   0.0248212   0.0585612

RMS errors: 


2×5 Matrix{Float64}:
 2.69785  2.66925  2.68337  2.69351  2.67653
 2.62905  2.74336  2.68749  2.64664  2.71465

### Model 5: GDP + Income

In [10]:
# Create the matrix for model 5
Amod5 = [ones(m) gdpClass incomeClass]

# Create a number storing the number of elements in each fold
fold = div(m,5); 

# Create a random permutation of m-values
I = Random.randperm(m);

# Storage for cross-validation results
xhats5 = zeros(3,5); 
rmsErrors5 = zeros(2,5);

# For each fold, compute the appropriate model and store its error.
anim = @animate for k = 1:5
    
    # Assign a random permutation into training and test data
    if (k == 1) 
        Itest = I[1 : fold]; 
        Itrain = I[fold+1 : end];
    elseif (k == 5)
        Itest = I[4 * fold+1 : end]; 
        Itrain = I[1:4 * fold];
    else 
        Itest = I[(k-1) * fold+1: k*fold];
        Itrain = I[[1 : (k-1) * fold ; k * fold + 1 : m]];
    end

    #display(Itest);
    #display(Itrain);
    
    # Compute sample sizes for training and test data
    mTest = length(Itest);
    mTrain = length(Itrain);

    # Compute the model based on training data and store it
    xhats5[:, k] = Amod5[Itrain, :] \ eduClass[Itrain];

    # Compute RMS errors for both training and test data
    rmsErrors5[1,k] = norm(Amod5[Itrain,:] * xhats5[:, k] - eduClass[Itrain]) / sqrt(mTrain);
    rmsErrors5[2,k] = norm(Amod5[Itest,:] * xhats5[:, k] - eduClass[Itest]) / sqrt(mTest);
    
    ## Plot actual (x) versus predicted (y) education
    # Plot test data
    tp5 = scatter(eduClass[Itest], Amod5[Itest, :] * xhats5[:, k], color="purple1", xlims=(0,25), ylims=(0,25), 
        title="Test data  " * L"(n=%$mTest)", xlabel="Actual education (years)", 
        ylabel="Predicted education (years)", minorgrid=true)
    plot!([0,25], [0,25], linestyle = :dash, linewidth=2, color="magenta") # y=x line
    
    # Plot training data
    p5 = scatter(eduClass[Itrain], Amod5[Itrain, :] * xhats5[:, k], color="purple1", xlims=(0,25), ylims=(0,25), 
        title="Training data  " * L"(n=%$mTrain)", xlabel="Actual education (years)", 
        ylabel="Predicted education (years)", minorgrid=true)
    plot!([0,25], [0,25], linestyle = :dash, linewidth=2, color="magenta") # y=x line

    plotModel5 = plot(tp5, p5, layout = (1, 2), legend=false, size=(1200, 620), margin=8mm,
        plot_title="Education Prediction using GDP and Income")
    annotate!(24, 1, [text("Fold " * string(k), 9)])
    savefig(plotModel5, "PlotModel_5f" * string(k) * ".png")
end

gif(anim, "PlotModel5.gif", fps=1)

println("x-hats: "); display(xhats5);
println("RMS errors: "); display(rmsErrors5);

x-hats: 


[ Info: Saved animation to C:\Users\bento\OneDrive\School Documents\Spring 2023\MAS 4106\Final Project\PlotModel5.gif


3×5 Matrix{Float64}:
 14.3052      14.3409      14.2447      14.3242      14.3585
  1.98742e-7   1.89202e-7   2.02127e-7   1.87509e-7   1.86577e-7
 -1.93692     -1.90436     -1.90738     -1.88107     -1.90022

RMS errors: 


2×5 Matrix{Float64}:
 2.53734  2.54345  2.55138  2.53215  2.53403
 2.5496   2.52485  2.49295  2.5701   2.56258

### Model 6: HDI & Income

In [11]:
# Create the matrix for model 6
Amod6 = [ones(m) marryMatrix natMatrix]

# Create a number storing the number of elements in each fold
fold = div(m,5); 

# Create a random permutation of m-values
I = Random.randperm(m);

# Storage for cross-validation results
xhats6 = zeros(6,5); 
rmsErrors6 = zeros(2,5);

# For each fold, compute the appropriate model and store its error.
anim = @animate for k = 1:5
    
    # Assign a random permutation into training and test data
    if (k == 1) 
        Itest = I[1 : fold]; 
        Itrain = I[fold+1 : end];
    elseif (k == 5)
        Itest = I[4 * fold+1 : end]; 
        Itrain = I[1:4 * fold];
    else 
        Itest = I[(k-1) * fold+1: k*fold];
        Itrain = I[[1 : (k-1) * fold ; k * fold + 1 : m]];
    end

    #display(Itest);
    #display(Itrain);
    
    # Compute sample sizes for training and test data
    mTest = length(Itest);
    mTrain = length(Itrain);

    # Compute the model based on training data and store it
    xhats6[:, k] = Amod6[Itrain, :] \ eduClass[Itrain];

    # Compute RMS errors for both training and test data
    rmsErrors6[1,k] = norm(Amod6[Itrain,:] * xhats6[:, k] - eduClass[Itrain]) / sqrt(mTrain);
    rmsErrors6[2,k] = norm(Amod6[Itest,:] * xhats6[:, k] - eduClass[Itest]) / sqrt(mTest);
    
    ## Plot actual (x) versus predicted (y) education
    # Plot test data
    tp6 = scatter(eduClass[Itest], Amod6[Itest, :] * xhats6[:, k], color="steelblue", xlims=(0,25), ylims=(0,25), 
        title="Test data  " * L"(n=%$mTest)", xlabel="Actual education (years)", 
        ylabel="Predicted education (years)", minorgrid=true)
    plot!([0,25], [0,25], linestyle = :dash, linewidth=2, color="magenta") # y=x line
    
    # Plot training data
    p6 = scatter(eduClass[Itrain], Amod6[Itrain, :] * xhats6[:, k], color="steelblue", xlims=(0,25), ylims=(0,25), 
        title="Training data  " * L"(n=%$mTrain)", xlabel="Actual education (years)", 
        ylabel="Predicted education (years)", minorgrid=true)
    plot!([0,25], [0,25], linestyle = :dash, linewidth=2, color="magenta") # y=x line

    plotModel6 = plot(tp6, p6, layout = (1, 2), legend=false, size=(1200, 620), margin=8mm,
        plot_title="Education Prediction using HDI and Income")
    annotate!(24, 1, [text("Fold " * string(k), 9)])
    savefig(plotModel6, "PlotModel_6f" * string(k) * ".png")
end

gif(anim, "PlotModel6.gif", fps=1)

println("x-hats: "); display(xhats6);
println("RMS errors: "); display(rmsErrors6);

x-hats: 


[ Info: Saved animation to C:\Users\bento\OneDrive\School Documents\Spring 2023\MAS 4106\Final Project\PlotModel6.gif


6×5 Matrix{Float64}:
 13.9546     13.9774     13.8956     13.8259    13.8632
  0.194544    0.179347    0.174809    0.171205   0.192378
  0.400461    0.397387    0.420958    0.395765   0.404041
 -0.0528477  -0.0623446   0.0155069   0.101406   0.050926
 -1.94426    -1.76576    -1.8603     -1.71956   -1.9093
  0.931865    0.945642    1.09564     1.1796     1.08529

RMS errors: 


2×5 Matrix{Float64}:
 2.69327  2.68054  2.68766  2.68049  2.683
 2.65211  2.70316  2.67469  2.70335  2.69328

## Education Prediction Models Factoring Controllable and Uncontrollable Attributes

### Model 7: Education Prediction using Uncontrollable Attributes

In [12]:
# Create the matrix for model 7
Amod7 = [ones(m) natMatrix gdpClass./1000 raceMatrix sexClass ageClass]

# Create a number storing the number of elements in each fold
fold = div(m,5); 

# Create a random permutation of m-values
I = Random.randperm(m);

# Storage for cross-validation results
xhats7 = zeros(11,5); 
rmsErrors7 = zeros(2,5);

# For each fold, compute the appropriate model and store its error.
anim = @animate for k = 1:5
    
    # Assign a random permutation into training and test data
    if (k == 1) 
        Itest = I[1 : fold]; 
        Itrain = I[fold+1 : end];
    elseif (k == 5)
        Itest = I[4 * fold+1 : end]; 
        Itrain = I[1:4 * fold];
    else 
        Itest = I[(k-1) * fold+1: k*fold];
        Itrain = I[[1 : (k-1) * fold ; k * fold + 1 : m]];
    end

    #display(Itest);
    #display(Itrain);
    
    # Compute sample sizes for training and test data
    mTest = length(Itest);
    mTrain = length(Itrain);

    # Compute the model based on training data and store it
    xhats7[:, k] = Amod7[Itrain, :] \ eduClass[Itrain];

    # Compute RMS errors for both training and test data
    rmsErrors7[1,k] = norm(Amod7[Itrain,:] * xhats7[:, k] - eduClass[Itrain]) / sqrt(mTrain);
    rmsErrors7[2,k] = norm(Amod7[Itest,:] * xhats7[:, k] - eduClass[Itest]) / sqrt(mTest);
    
    ## Plot actual (x) versus predicted (y) education
    # Plot test data
    tp7 = scatter(eduClass[Itest], Amod7[Itest, :] * xhats7[:, k], color="green1", xlims=(0,25), ylims=(0,25), 
        title="Test data  " * L"(n=%$mTest)", xlabel="Actual education (years)", 
        ylabel="Predicted education (years)", minorgrid=true)
    plot!([0,25], [0,25], linestyle = :dash, linewidth=2, color="magenta") # y=x line
    
    # Plot training data
    p7 = scatter(eduClass[Itrain], Amod7[Itrain, :] * xhats7[:, k], color="green1", xlims=(0,25), ylims=(0,25), 
        title="Training data  " * L"(n=%$mTrain)", xlabel="Actual education (years)", 
        ylabel="Predicted education (years)", minorgrid=true)
    plot!([0,25], [0,25], linestyle = :dash, linewidth=2, color="magenta") # y=x line

    plotModel7 = plot(tp7, p7, layout = (1, 2), legend=false, size=(1200, 620), margin=8mm,
        plot_title="Education Prediction using Uncontrollable Attributes")
    annotate!(24, 1, [text("Fold " * string(k), 9)])
    savefig(plotModel7, "PlotModel_7f" * string(k) * ".png")
end

gif(anim, "PlotModel7.gif", fps=1)

println("x-hats: "); display(xhats7);
println("RMS errors: "); display(rmsErrors7);

x-hats: 


[ Info: Saved animation to C:\Users\bento\OneDrive\School Documents\Spring 2023\MAS 4106\Final Project\PlotModel7.gif


11×5 Matrix{Float64}:
 13.042        12.8421       12.9898       12.9521       12.9683
 -2.72529      -2.54823      -2.78034      -2.54058      -2.71554
 -1.54074      -1.20605      -1.31554      -1.28039      -1.42323
  0.915771      1.05491       1.04218       0.960519      0.844973
  0.000450635   0.000442316   0.000469148   0.000448919   0.000443751
  0.531306      0.627844      0.566011      0.484308      0.656186
  0.877301      0.959581      0.81059       0.851015      1.0489
 -0.165588     -0.0937166    -0.282321     -0.282498     -0.0884536
 -0.0172389     0.049544     -0.0360075    -0.146895      0.0813923
  0.02444       0.0579783     0.0472074     0.0487305     0.0593033
  0.00543944    0.00496285    0.00380135    0.00461626    0.00456561

RMS errors: 


2×5 Matrix{Float64}:
 2.59927  2.61504  2.61426  2.60971  2.61775
 2.65951  2.59675  2.60041  2.61835  2.58593

### Model 8: Education Prediction using Controllable Attributes

In [13]:
# Create the matrix for model 8
Amod8 = [ones(m) workClassMatrix occMatrix marryMatrix incomeClass hrsPerWeekClass]

# Create a number storing the number of elements in each fold
fold = div(m,5); 

# Create a random permutation of m-values
I = Random.randperm(m);

# Storage for cross-validation results
xhats8 = zeros(12,5); 
rmsErrors8 = zeros(2,5);

# For each fold, compute the appropriate model and store its error.
anim = @animate for k = 1:5
    
    # Assign a random permutation into training and test data
    if (k == 1) 
        Itest = I[1 : fold]; 
        Itrain = I[fold+1 : end];
    elseif (k == 5)
        Itest = I[4 * fold+1 : end]; 
        Itrain = I[1:4 * fold];
    else 
        Itest = I[(k-1) * fold+1: k*fold];
        Itrain = I[[1 : (k-1) * fold ; k * fold + 1 : m]];
    end

    #display(Itest);
    #display(Itrain);
    
    # Compute sample sizes for training and test data
    mTest = length(Itest);
    mTrain = length(Itrain);

    # Compute the model based on training data and store it
    xhats8[:, k] = Amod8[Itrain, :] \ eduClass[Itrain];

    # Compute RMS errors for both training and test data
    rmsErrors8[1,k] = norm(Amod8[Itrain,:] * xhats8[:, k] - eduClass[Itrain]) / sqrt(mTrain);
    rmsErrors8[2,k] = norm(Amod8[Itest,:] * xhats8[:, k] - eduClass[Itest]) / sqrt(mTest);
    
    ## Plot actual (x) versus predicted (y) education
    # Plot test data
    tp8 = scatter(eduClass[Itest], Amod8[Itest, :] * xhats8[:, k], color="darkturquoise", xlims=(0,25), ylims=(0,25), 
        title="Test data  " * L"(n=%$mTest)", xlabel="Actual education (years)", 
        ylabel="Predicted education (years)", minorgrid=true)
    plot!([0,25], [0,25], linestyle = :dash, linewidth=2, color="magenta") # y=x line
    
    # Plot training data
    p8 = scatter(eduClass[Itrain], Amod8[Itrain, :] * xhats8[:, k], color="darkturquoise", xlims=(0,25), ylims=(0,25), 
        title="Training data  " * L"(n=%$mTrain)", xlabel="Actual education (years)", 
        ylabel="Predicted education (years)", minorgrid=true)
    plot!([0,25], [0,25], linestyle = :dash, linewidth=2, color="magenta") # y=x line

    plotModel8 = plot(tp8, p8, layout = (1, 2), legend=false, size=(1200, 620), margin=8mm,
        plot_title="Education Prediction using Controllable Attributes")
    annotate!(24, 1, [text("Fold " * string(k), 9)])
    savefig(plotModel8, "PlotModel_8f" * string(k) * ".png")
    
end

gif(anim, "PlotModel8.gif", fps=1)

println("x-hats: "); display(xhats8);
println("RMS errors: "); display(rmsErrors8);

x-hats: 


[ Info: Saved animation to C:\Users\bento\OneDrive\School Documents\Spring 2023\MAS 4106\Final Project\PlotModel8.gif


12×5 Matrix{Float64}:
 13.66       13.9326     13.7272      13.4756     13.2811
  0.0269376  -0.159536   -0.00540928   0.175995    0.377059
  0.206359    0.0220225   0.19483      0.331483    0.571881
  1.12609     0.92145     1.05968      1.27457     1.46885
  0.35586     0.312465    0.355668     0.389984    0.292318
  1.44653     1.34716     1.42166      1.44211     1.37869
  1.68272     1.60611     1.6794       1.72613     1.63554
 -0.123143   -0.21425    -0.138179    -0.12817    -0.19682
  0.449082    0.465503    0.440574     0.462135    0.475685
 -0.325552   -0.281961   -0.333268    -0.306792   -0.295237
 -1.71803    -1.71452    -1.73585     -1.69998    -1.70044
  0.0151695   0.0145896   0.0153316    0.0151112   0.0163543

RMS errors: 


2×5 Matrix{Float64}:
 2.3983   2.39271  2.40507  2.40519  2.40855
 2.41713  2.43938  2.39013  2.3897   2.37617

## Human Development Index Prediction Models Factoring Controllable and Uncontrollable Attributes

### Model 9: HDI Prediction using Uncontrollable Attributes

In [14]:
# Create the matrix for model 9
Amod9 = [ones(m) natMatrix gdpClass./1000 raceMatrix sexClass ageClass]

# Create a number storing the number of elements in each fold
fold = div(m,5); 

# Create a random permutation of m-values
I = Random.randperm(m);

# Storage for cross-validation results
xhats9 = zeros(11,5); 
rmsErrors9 = zeros(2,5);

# For each fold, compute the appropriate model and store its error.
anim = @animate for k = 1:5
    
    # Assign a random permutation into training and test data
    if (k == 1) 
        Itest = I[1 : fold]; 
        Itrain = I[fold+1 : end];
    elseif (k == 5)
        Itest = I[4 * fold+1 : end]; 
        Itrain = I[1:4 * fold];
    else 
        Itest = I[(k-1) * fold+1: k*fold];
        Itrain = I[[1 : (k-1) * fold ; k * fold + 1 : m]];
    end

    #display(Itest);
    #display(Itrain);
    
    # Compute sample sizes for training and test data
    mTest = length(Itest);
    mTrain = length(Itrain);

    # Compute the model based on training data and store it
    xhats9[:, k] = Amod9[Itrain, :] \ hdiClass[Itrain];

    # Compute RMS errors for both training and test data
    rmsErrors9[1,k] = norm(Amod9[Itrain,:] * xhats9[:, k] - hdiClass[Itrain]) / sqrt(mTrain);
    rmsErrors9[2,k] = norm(Amod9[Itest,:] * xhats9[:, k] - hdiClass[Itest]) / sqrt(mTest);
    
    ## Plot actual (x) versus predicted (y) HDI
    # Plot test data
    tp9 = scatter(hdiClass[Itest], Amod9[Itest, :] * xhats9[:, k], color="tomato", xlims=(0,1), ylims=(0,1), 
        title="Test data  " * L"(n=%$mTest)", xlabel="Actual HDI", 
        ylabel="Predicted HDI", minorgrid=true)
    plot!([0,1], [0,1], linestyle = :dash, linewidth=2, color="magenta") # y=x line
    
    # Plot training data
    p9 = scatter(hdiClass[Itrain], Amod9[Itrain, :] * xhats9[:, k], color="tomato", xlims=(0,1), ylims=(0,1), 
        title="Training data  " * L"(n=%$mTrain)", xlabel="Actual HDI", 
        ylabel="Predicted HDI", minorgrid=true)
    plot!([0,1], [0,1], linestyle = :dash, linewidth=2, color="magenta") # y=x line

    plotModel9 = plot(tp9, p9, layout = (1, 2), legend=false, size=(1200, 620), margin=8mm,
        plot_title="HDI Prediction using Uncontrollable Attributes")
    annotate!(0.96, 0.04, [text("Fold " * string(k), 9)])
    savefig(plotModel9, "PlotModel_9f" * string(k) * ".png")
    
end

gif(anim, "PlotModel9.gif", fps=1)

println("x-hats: "); display(xhats9);
println("RMS errors: "); display(rmsErrors9);

x-hats: 


[ Info: Saved animation to C:\Users\bento\OneDrive\School Documents\Spring 2023\MAS 4106\Final Project\PlotModel9.gif


11×5 Matrix{Float64}:
  0.734605      0.731388     0.735059      0.73159       0.732135
 -0.109331     -0.10631     -0.108306     -0.10596      -0.108357
 -0.162632     -0.163177    -0.165195     -0.162463     -0.165344
 -0.176781     -0.173715    -0.175805     -0.174509     -0.175466
  3.33779e-5    3.28499e-5   3.30788e-5    3.28604e-5    3.31983e-5
  0.00261042    0.00661642   0.00329476    0.00611266    0.00555231
 -0.0068521    -0.00377317  -0.00513777   -0.00350446   -0.00411044
  0.00351155    0.00787404   0.00429342    0.00726691    0.00628086
 -0.00155952    0.00256591  -0.000921422   0.00199949    0.00177733
  0.000928655   0.00103772   0.000870598   0.000744114   0.000824427
  3.55595e-5    3.59166e-5   3.61952e-5    3.46453e-5    3.23806e-5

RMS errors: 


2×5 Matrix{Float64}:
 0.0242693  0.0245628  0.0244332  0.0244039  0.0244891
 0.0251116  0.0239289  0.0244464  0.0245669  0.0242223

### Model 10: HDI Prediction using Uncontrollable Attributes

In [15]:
# Create the matrix for model 10
Amod10 = [ones(m) workClassMatrix occMatrix marryMatrix incomeClass hrsPerWeekClass]

# Create a number storing the number of elements in each fold
fold = div(m,5); 

# Create a random permutation of m-values
I = Random.randperm(m);

# Storage for cross-validation results
xhats10 = zeros(12,5); 
rmsErrors10 = zeros(2,5);

# For each fold, compute the appropriate model and store its error.
anim = @animate for k = 1:5
    
    # Assign a random permutation into training and test data
    if (k == 1) 
        Itest = I[1 : fold]; 
        Itrain = I[fold+1 : end];
    elseif (k == 5)
        Itest = I[4 * fold+1 : end]; 
        Itrain = I[1:4 * fold];
    else 
        Itest = I[(k-1) * fold+1: k*fold];
        Itrain = I[[1 : (k-1) * fold ; k * fold + 1 : m]];
    end

    #display(Itest);
    #display(Itrain);
    
    # Compute sample sizes for training and test data
    mTest = length(Itest);
    mTrain = length(Itrain);

    # Compute the model based on training data and store it
    xhats10[:, k] = Amod10[Itrain, :] \ hdiClass[Itrain];

    # Compute RMS errors for both training and test data
    rmsErrors10[1,k] = norm(Amod10[Itrain,:] * xhats10[:, k] - hdiClass[Itrain]) / sqrt(mTrain);
    rmsErrors10[2,k] = norm(Amod10[Itest,:] * xhats10[:, k] - hdiClass[Itest]) / sqrt(mTest);
    
    ## Plot actual (x) versus predicted (y) HDI
    # Plot test data
    tp10 = scatter(hdiClass[Itest], Amod10[Itest, :] * xhats10[:, k], color="goldenrod", xlims=(0,1), ylims=(0,1), 
        title="Test data  " * L"(n=%$mTest)", xlabel="Actual HDI", 
        ylabel="Predicted HDI", minorgrid=true)
    plot!([0,1], [0,1], linestyle = :dash, linewidth=2, color="magenta") # y=x line
    
    # Plot training data
    p10 = scatter(hdiClass[Itrain], Amod10[Itrain, :] * xhats10[:, k], color="goldenrod", xlims=(0,1), ylims=(0,1), 
        title="Training data  " * L"(n=%$mTrain)", xlabel="Actual HDI", 
        ylabel="Predicted HDI", minorgrid=true)
    plot!([0,1], [0,1], linestyle = :dash, linewidth=2, color="magenta") # y=x line

    plotModel10 = plot(tp10, p10, layout = (1, 2), legend=false, size=(1200, 620), margin=8mm,
        plot_title="HDI Prediction using Controllable Attributes")
    annotate!(24/25, 1/25, [text("Fold " * string(k), 9)])
    savefig(plotModel10, "PlotModel_10f" * string(k) * ".png")
    
end

gif(anim, "PlotModel10.gif", fps=1)

println("x-hats: "); display(xhats10);
println("RMS errors: "); display(rmsErrors10);

x-hats: 


[ Info: Saved animation to C:\Users\bento\OneDrive\School Documents\Spring 2023\MAS 4106\Final Project\PlotModel10.gif


12×5 Matrix{Float64}:
  0.861265     0.860942     0.862458     0.869652      0.877466
 -0.00947663  -0.00920297  -0.0100243   -0.0117254    -0.0255398
 -0.00360414  -0.00397285  -0.00425637  -0.00650393   -0.0201163
 -0.00121374  -3.9252e-5   -0.00104057  -0.00372292   -0.0171936
  0.00499996   0.00373109   0.00383222  -0.00113927    0.00396805
  0.015298     0.0146859    0.0154276    0.00914442    0.015148
  0.00959016   0.00903487   0.0108229    0.00467162    0.010078
  0.00565312   0.00538433   0.00521983   0.000393631   0.00587724
 -0.005171    -0.0050621   -0.00467495  -0.00513698   -0.00546417
 -0.0111575   -0.0109958   -0.0120208   -0.011551     -0.0112966
 -0.00746344  -0.00768521  -0.00839229  -0.00827635   -0.00738626
  6.00894e-5   7.13248e-5   6.40136e-5   6.6668e-5     6.37291e-5

RMS errors: 


2×5 Matrix{Float64}:
 0.069981   0.0700954  0.0697481  0.0698604  0.0695274
 0.0693078  0.068842   0.0702504  0.0698064  0.0711094